In [3]:
import numpy as np
import pandas as pd
from os import listdir
import cv2
from keras_preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.metrics import categorical_crossentropy
from keras.optimizers import SGD

2022-11-10 02:47:28.532842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def image_array(image_dir):
    image_size = tuple((124, 124))

    image = cv2.imread(image_dir)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    if image is not None :
        image = cv2.resize(image, image_size)   
        return img_to_array(image)
    else :
        return np.array([])


In [53]:
data_dir = "../Plant_leave_diseases_dataset_without_augmentation"
labels = listdir(data_dir)

In [5]:

labels_numbers = [i for i in range(0, len(labels))]
images_list = []
labels_list = []
images_list_test = []
labels_list_test = []
if ".DS_Store" in labels:
    labels.remove(".DS_Store")
for i in range(0, len(labels)):
    print(i)
    label_dir = labels[i]
    cur_images_list = listdir(f"{data_dir}/{label_dir}")
    if ".DS_Store" in cur_images_list:
        cur_images_list.remove(".DS_Store")
    image_len = len(cur_images_list)
    print(image_len)

    split_point = int(image_len * 0.8)
    for img in cur_images_list[:split_point]:
#         print(f"{data_dir}/{label_dir}/{img}")
        images_list.append(image_array(f"{data_dir}/{label_dir}/{img}"))
        labels_list.append(i)
    for img in cur_images_list[split_point:]:
        
        images_list_test.append(image_array(f"{data_dir}/{label_dir}/{img}"))
        labels_list_test.append(i)


0
1000
1
1000
2
513
3
985
4
1192
5
1162
6
152


In [8]:
Xtrain = np.array(images_list)
Ytrain = np.array(labels_list)
Xtest = np.array(images_list_test)
Ytest = np.array(labels_list_test)

In [9]:
Xtrain.shape



(4801, 124, 124, 3)

In [10]:


Xtrain = Xtrain.reshape(Xtrain.shape[0], 124, 124, 3)
Xtest = Xtest.reshape(Xtest.shape[0], 124, 124, 3)
Xtrain = np.pad(Xtrain, ((0,0),(2,2),(2,2),(0,0)), 'constant')
Xtest = np.pad(Xtest, ((0,0),(2,2),(2,2),(0,0)), 'constant')
mean_px = Xtrain.mean().astype(np.float32)
std_px = Xtrain.std().astype(np.float32)
Xtrain = (Xtrain - mean_px)/(std_px)


In [11]:
print(len(images_list))

4801


In [12]:
Xtrain.shape

(4801, 128, 128, 3)

In [13]:
Ytrain.shape

(4801,)

In [14]:
# from keras.utils.np_utils import to_categorical
# Ytrain = to_categorical(Ytrain)

In [15]:
    
# Initialize the model
model = Sequential()
#Layer 1
#C$onv Lay
model.add(Conv2D(filters = 6, 
                 kernel_size = 5, 
                 strides = 1, 
                 activation = 'relu', 
                 input_shape = (128,128,3)))
#Pooling layer 1
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Layer 2
#Conv Layer 2
model.add(Conv2D(filters = 16, 
                 kernel_size = 5,
                 strides = 1,
                 activation = 'relu',
                 input_shape = (124,124,6)))
#Pooling Layer 2
model.add(MaxPooling2D(pool_size = 2, strides = 2))
#Flatten
model.add(Flatten())
#Layer 3
#Fully connected layer 1
model.add(Dense(units = 120, activation = 'relu'))
#Layer 4
#Fully connected layer 2
model.add(Dense(units = 84, activation = 'relu'))
#Layer 5
#Output Layer
model.add(Dense(units = 1, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


2022-11-10 02:48:37.023614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 6)       456       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 58, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 29, 29, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 13456)             0         
                                                                 
 dense (Dense)               (None, 120)               1

In [17]:
len(images_list)

4801

In [18]:
Xtrain.shape

(4801, 128, 128, 3)

In [19]:
Ytrain.shape

(4801,)

In [20]:
# from keras.utils.np_utils import to_categorical
# Ytrain = to_categorical(Ytrain)
model.fit(Xtrain ,Ytrain, steps_per_epoch = 10, epochs = 10)
predictions = model.predict(Xtest)


Epoch 1/10
10/10 [==============================] - 36s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 2/10
10/10 [==============================] - 33s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 3/10
10/10 [==============================] - 31s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 4/10
10/10 [==============================] - 31s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 5/10
10/10 [==============================] - 32s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 6/10
10/10 [==============================] - 32s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 7/10
10/10 [==============================] - 34s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 8/10
10/10 [==============================] - 32s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 9/10
10/10 [==============================] - 33s 3s/step - loss: 0.0000e+00 - accuracy: 0.1666
Epoch 10/10
38/38 [==============================] - 5s 112ms/step


In [21]:
Ytest

array([0, 0, 0, ..., 6, 6, 6])

In [22]:
len(Ytest)

1203

In [23]:
predictions

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [24]:
labels[1]

'Potato___Late_blight'

In [25]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report
#metrics
disease_accuracy = accuracy_score(Ytest, predictions)
disease_recall = recall_score(Ytest, predictions, average='weighted')
disease_f1 = f1_score(Ytest, predictions, average='weighted')
disease_precision = precision_score(Ytest, predictions, average='weighted')

print("accuracy: ",disease_accuracy)
print("recall: ",disease_recall)
print("f1: ",disease_f1)
print("precision: ",disease_precision)
print(classification_report(Ytest, predictions))

accuracy:  0.1662510390689942
recall:  0.1662510390689942
f1:  0.04739872817362629
precision:  0.027639407991520232
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       200
           1       0.17      1.00      0.29       200
           2       0.00      0.00      0.00       103
           3       0.00      0.00      0.00       197
           4       0.00      0.00      0.00       239
           5       0.00      0.00      0.00       233
           6       0.00      0.00      0.00        31

    accuracy                           0.17      1203
   macro avg       0.02      0.14      0.04      1203
weighted avg       0.03      0.17      0.05      1203



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: 

In [26]:
import pickle
disease_model_file = open('disease_model.pkl', 'wb')
pickle.dump(model, disease_model_file)

INFO:tensorflow:Assets written to: ram://a62a60cc-4722-4a0c-9ab2-15c097f44ebc/assets


INFO:tensorflow:Assets written to: ram://a62a60cc-4722-4a0c-9ab2-15c097f44ebc/assets


In [27]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import time
import json

import pickle
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image

/Users/yuejiao/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def feature_extract(image_dir):
    vgg_model = models.vgg19(pretrained=True)
    new_classifier = torch.nn.Sequential(*list(vgg_model.children())[-1][:6])
    new_classifier.add_module("Linear output", torch.nn.Linear(4096, 256))
    vgg_model.classifier = new_classifier
    trans = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    im = Image.open(image_dir).convert('RGB')
    im = trans(im)
    im.unsqueeze_(dim=0)
    vgg_model = vgg_model.eval()
    feature_extracted = vgg_model(im).data.numpy().tolist()[0]
    feature_extracted = [float(i) for i in feature_extracted]
    return feature_extracted

In [ ]:
data_dir = "../Plant_leave_diseases_dataset_without_augmentation"
labels = listdir(data_dir)

In [55]:
labels

['Potato___Early_blight',
 '.DS_Store',
 'Potato___Late_blight',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Northern_Leaf_Blight',
 'Corn___Common_rust',
 'Corn___healthy',
 'Potato___healthy']

In [33]:

labels_numbers = [i for i in range(0, len(labels))]
images_list = []
labels_list = []

if ".DS_Store" in labels:
    labels.remove(".DS_Store")
for i in range(0, len(labels)):
    print(i)
    
    label_dir = labels[i]
    cur_images_list = listdir(f"{data_dir}/{label_dir}")
    if ".DS_Store" in cur_images_list:
        cur_images_list.remove(".DS_Store")
    image_len = len(cur_images_list)
    print(image_len)

    count = 0
    for img in cur_images_list:
        if count % 100 == 0:
            print(count)
        count += 1
#         print(f"{data_dir}/{label_dir}/{img}")
        images_list.append(feature_extract(f"{data_dir}/{label_dir}/{img}"))
        labels_list.append(i)



0
1000
0
100
200
300
400
500
600
700
800
900
1
1000
0
100
200
300
400
500
600
700
800
900
2
513
0
100
200
300
400
500
3
985
0
100
200
300
400
500
600
700
800
900
4
1192
0
100
200
300
400
500
600
700
800
900
1000
1100
5
1162
0
100
200
300
400
500
600
700
800
900
1000
1100
6
152
0
100


In [41]:
images_list = np.array(images_list)
labels_list = np.array([labels_list])

In [42]:
len(images_list)

6004

In [43]:
labels_list

array([[0, 0, 0, ..., 6, 6, 6]])

In [44]:
# pca reduction

# pca = PCA(n_components=10)
# pca_reduced = pca.fit_transform(images_list)


#merge two array
merged_features = np.concatenate((labels_list.T,images_list), 1)
image_columns = ['image_'+str(i) for i in range(0, 256)]
label_columns = ['label'] 
merged_colums = label_columns + image_columns
print(merged_features.shape)

print(len(merged_colums))
df = pd.DataFrame(data = merged_features, columns=merged_colums)


df

(6004, 257)
257


,label,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,...,image_246,image_247,image_248,image_249,image_250,image_251,image_252,image_253,image_254,image_255
0,0.0,0.333085,0.106030,0.118983,-0.136160,0.345141,0.199062,-0.056617,-0.084141,-0.007428,...,-0.264285,-0.542027,-0.160324,0.008155,-0.525646,-0.038651,-0.296421,-0.256159,-0.000710,-0.346456
1,0.0,-0.012369,-0.423730,-0.011569,0.469790,-0.036361,-0.203854,0.204341,-0.408809,-0.718812,...,-0.002708,-0.140796,-0.145836,0.136490,0.041334,0.225173,-0.260967,0.338846,-0.172610,-0.048522
2,0.0,-0.078077,-0.488927,0.190697,0.229692,0.055479,0.344402,-0.084533,-0.399928,0.496378,...,0.522652,0.202150,0.085541,-0.044599,-0.171629,0.405004,0.076145,0.136882,0.151807,-0.431959
3,0.0,-0.586274,-0.028080,-0.613123,0.097704,0.242799,0.173298,0.714489,-0.726794,-0.136778,...,0.105646,0.665691,-0.906582,0.145873,-0.032639,0.346668,-0.140202,-0.681944,-0.264344,-0.252501
4,0.0,-0.621480,0.061952,-0.261676,-0.189272,-0.489902,-0.232653,-0.058048,-0.125082,-0.146416,...,0.215906,-0.139131,-0.146766,-0.129132,0.127822,0.288117,0.023322,-0.371559,0.167056,-0.084310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999,6.0,-0.202717,0.337898,0.041325,-0.468199,-0.094142,0.055987,-0.092150,-0.430311,0.170651,...,-0.095759,0.215642,0.108238,-0.108072,-0.162908,0.026961,-0.077680,-0.322685,0.520305,0.108127
6000,6.0,0.577732,-0.078937,0.618071,-0.174721,-0.170915,-0.274375,0.079242,0.130200,0.172056,...,0.265790,-0.220983,-0.271514,-0.425752,-0.580750,0.198537,0.331740,0.070441,0.098987,-0.278149
6001,6.0,0.282249,-0.390429,0.076814,-0.259895,-0.326229,0.326043,0.511024,-0.137138,-0.169261,...,-0.162045,-0.274111,-0.406557,-0.184440,0.190075,0.328766,-0.272904,0.207780,0.026285,-0.335218
6002,6.0,0.016297,0.135664,-0.140099,-0.097214,-0.295706,-0.383177,-0.102116,0.240893,0.080595,...,-0.034478,-0.000823,-0.351963,-0.351104,-0.106051,0.377987,0.096716,-0.187763,0.464881,0.053844


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

feature_name = ['image_'+str(i) for i in range(0,256)]
features = df[feature_name]
target = df['label']
labels = df['label']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features,target,test_size = 0.2)
svm = SVC(gamma='scale')

svm.fit(Xtrain,Ytrain)

svm_predictions = svm.predict(Xtest)

svm_accuracy = accuracy_score(Ytest, svm_predictions)
svm_recall = recall_score(Ytest, svm_predictions, average='weighted')
svm_f1 = f1_score(Ytest, svm_predictions, average='weighted')
svm_precision = precision_score(Ytest, svm_predictions, average='weighted')

print("accuracy: ",svm_accuracy)
print("recall: ",svm_recall)
print("f1: ",svm_f1)
print("precision: ",svm_precision)
print(classification_report(Ytest, svm_predictions))

accuracy:  0.33305578684429643
recall:  0.33305578684429643
f1:  0.30372246104268097
precision:  0.29153953014414064
              precision    recall  f1-score   support

         0.0       0.48      0.43      0.45       224
         1.0       0.23      0.18      0.20       194
         2.0       0.00      0.00      0.00       100
         3.0       0.23      0.18      0.20       198
         4.0       0.26      0.38      0.31       230
         5.0       0.41      0.64      0.50       230
         6.0       0.00      0.00      0.00        25

    accuracy                           0.33      1201
   macro avg       0.23      0.26      0.24      1201
weighted avg       0.29      0.33      0.30      1201



/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: 

In [51]:
#save models
svm_disease_file = open('svm_disease_model.pkl', 'wb')
pickle.dump(svm, svm_disease_file)
svm_disease_file.close()